In [1]:
import re
import json
import requests
import unicodedata
from pathlib import Path
from bs4 import BeautifulSoup

In [2]:
#=================================================#
# URL List
#=================================================#
url_list = [
  'https://www.uta-net.com/artist/12550/',       #乃木坂46
  'https://www.uta-net.com/artist/22653/',       #ヨルシカ
  'https://www.uta-net.com/artist/28370/'        #YOASOBI
]

for parent_url in url_list:

  json_data = None

  response = requests.get(parent_url, timeout=10)
  soup = BeautifulSoup(response.text, 'html.parser')

#=================================================#
# Artist Name
#=================================================#
  elements = soup.find_all('script', attrs={'type': 'text/javascript'})

  for element in elements:
    json_data = element.string or element.text
    if ('artist_name' in json_data):
      break

  json_data = re.search(r'microadTd\.UN\.start\(\s*({.*?})\s*\)', json_data, re.DOTALL).group(1)
  json_data = json.loads(json_data)
  artist_name = json_data['artist_name']

#================================================= #
# Page Count
#================================================= #
  element = soup.find('div', class_='col-7 col-lg-3 text-start text-lg-end d-none d-lg-block')
  page_count = re.search(r'全([0-9０-９]+)ページ中', element.get_text(strip=True)).group(1)
  
#================================================= #
# Song Number, Song Title Song lyric
#================================================= #
  song_elements = []

  for page in range(int(page_count)):

    child_url = f'{parent_url}0/{page + 1}/'
    response = requests.get(child_url, timeout=10)
    soup = BeautifulSoup(response.text, 'html.parser')

    song_ids = soup.find_all('a', class_='py-2 py-lg-0')                                      # 曲番号
    song_titles = soup.find_all('span', class_='fw-bold songlist-title pb-1 pb-lg-0')         # 曲名
    song_lyrics = soup.find_all('span', class_='d-block d-lg-none utaidashi text-truncate')   # 歌詞

    for song_id, song_title, song_lyric in zip(song_ids, song_titles, song_lyrics):
      song_elements.append({
        'id':    re.search(r'/song/(\d+)/', song_id['href']).group(1),
        'title': song_title.get_text(strip=True),
        'lyric': song_lyric.get_text(strip=True),
      })

#================================================= #
# Write File
#================================================= #
  artist_dir = Path('./') / artist_name
  artist_dir.mkdir(parents=True, exist_ok=True)

  for song_element in song_elements:
    id = song_element["id"]
    title = re.sub(r'[\\/:*?"<>|]', "_", unicodedata.normalize("NFC", song_element["title"]))
    file_name  = id + '_' + title + '.txt'
    file_path = artist_dir / file_name
    file_path.write_text(song_element['lyric'], encoding='utf-8')## 